In [ ]:
import json
import sys 
sys.path.append('..')
import sqlite3
import pandas as pd
import numpy as np

def table_to_df(df):
    # Transformation de la table SQLite en dataframe en parsant l'arbre JSON
    # Return un dictionnaire et une liste de colonnes (les features histo)
    tree_as_dict = {}
    features_col = []
    for index, row in df.iterrows():
        tree_as_dict.setdefault("id", []).append(row[0])
        tree_as_dict.setdefault("patient_id", []).append(row[1])
        tree_as_dict.setdefault("expert_id", []).append(row[2])
        tree_as_dict.setdefault("biopsie_id", []).append(row[3])
        tree_as_dict.setdefault("muscle_prelev", []).append(row[4])
        tree_as_dict.setdefault("age_biopsie", []).append(row[5])
        tree_as_dict.setdefault("date_envoie", []).append(row[6])
        tree_as_dict.setdefault("gene_diag", []).append(row[7])
        tree_as_dict.setdefault("comment", []).append(row[9])
        tree_as_dict.setdefault("conclusion", []).append(row[10])
        tree_as_dict.setdefault("datetime", []).append(row[11])

        my_tree = json.loads(row[8])
        for feature in my_tree:
            tree_as_dict.setdefault(feature["text"], []).append(
                float(feature["data"].get("presence", -0.25))
            )
            if index == 0:
                features_col.append(feature["text"])
    df_return = pd.DataFrame.from_dict(tree_as_dict)
    return df_return, features_col


def db_to_df():
    con = sqlite3.connect("../app.db")
    df = pd.read_sql_query("SELECT * from report_histo", con)
    return df


def process_df(df):
    df = df.replace(
        {
            "COM_CCD": "COM",
            "COM_MMM": "COM",
            "NM_CAP": "NM",
            "CFTD": "OTHER",
            "NON_CM": "OTHER",
            "CM": "UNCLEAR",
        }
    )
    df = df.replace({-0.25: np.nan, 0.25: 1, 0.5: 1, 0.75: 1})
    return df
df = db_to_df()
df, features_col = table_to_df(df)
df = process_df(df)

In [ ]:
with open("../config/ontology.json", "r") as fp:
    onto = json.load(fp)

In [ ]:
# n_gene = df["gene_diag"].value_counts()
# n_phenotype = df["conclusion"].value_counts()
# index_gene = n_gene.index
# index_phenotype = n_phenotype.index

In [ ]:
for term in onto:
    df_temp = df[df[term["text"]]==1]
    gene_datamined_temp = list(df_temp["gene_diag"].value_counts().index)
    phenotype_datamined_temp = list(df_temp["conclusion"].value_counts().index)
    if gene_datamined_temp == []:
        term["data"]["gene_datamined"] = ""
    else:
        term["data"]["gene_datamined"] = ",".join(gene_datamined_temp)
    if phenotype_datamined_temp == []:
        term["data"]["phenotype_datamined"] = ""
    else:
        term["data"]["phenotype_datamined"] = ",".join(phenotype_datamined_temp)

In [ ]:
with open("../config/ontology.json", "w") as fp:
    json.dump(onto, fp, indent=4)